In [1]:
import os

ROOT_DATA = os.environ.get('LARD_DATA_ROOT_PATH')
ROOT_PROJECT = os.environ.get('LARD_PROJECT_ROOT_PATH')

print(ROOT_DATA)
print(ROOT_PROJECT)

print('HELLO')


/home/yeli/yeli/data/lard
/home/yeli/Nextcloud/lard/lard-detection
HELLO


# 查看已有数据集

In [1]:
import fiftyone as fo
all_datasets = fo.list_datasets()
print(all_datasets)

['lard_test_real_edge', 'lard_test_real_nominal', 'lard_test_synth', 'lard_train', 'lard_train_all', 'lard_val']


# 打开web界面查看结果

In [2]:
dataset = fo.load_dataset("lard_test_synth")
print(dataset.get_field_schema().keys())
session = fo.launch_app(dataset, auto=False)

odict_keys(['id', 'filepath', 'tags', 'metadata', 'ground_truth_detections', 'predictions_yolov3_train_val', 'predictions_yolov8n-p2_train_val'])

Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


In [3]:
session.close()

# 删除全部预测

In [3]:
from src.utils.database import delete_all_predictions
    
for name in all_datasets:
    dataset = fo.load_dataset(name)
    delete_all_predictions(dataset)


删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []
删除字段: []


In [1]:
from src.utils.database import eval_detections_in_database

all_results = eval_detections_in_database('yolov8n-p2_train_val')

# todo: 写一个函数，自动处理各数据集/各模型 进行评估

正在评估: test_synth
Evaluating detections...
 100% |███████████████| 2212/2212 [4.5s elapsed, 0s remaining, 500.3 samples/s]      
Performing IoU sweep...
 100% |███████████████| 2212/2212 [4.8s elapsed, 0s remaining, 473.9 samples/s]      
              precision    recall  f1-score   support

      runway       0.97      0.97      0.97      2212

   micro avg       0.97      0.97      0.97      2212
   macro avg       0.97      0.97      0.97      2212
weighted avg       0.97      0.97      0.97      2212

mAP: 0.715
   accuracy  precision  recall  fscore  support
0      0.94      0.971   0.968   0.969     2212
正在评估: test_real_nominal
Evaluating detections...
 100% |███████████████| 1500/1500 [3.0s elapsed, 0s remaining, 507.9 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1500/1500 [3.2s elapsed, 0s remaining, 478.7 samples/s]      
              precision    recall  f1-score   support

      runway       0.86      0.78      0.82      1500

   micro avg       0.86    

In [3]:
for datatype, results in all_results.items():
    plot_pr = results.plot_pr_curves()
    display(plot_pr)

FigureWidget({
    'data': [{'customdata': array([0.98346 , 0.959411, 0.952703, 0.948933, 0.945987, 0.943281, 0.940937,
                                   0.938159, 0.935524, 0.933398, 0.931047, 0.926193, 0.839753, 0.838682,
                                   0.837822, 0.836778, 0.835307, 0.833886, 0.832553, 0.831275, 0.830008,
                                   0.828706, 0.827584, 0.826041, 0.824877, 0.823341, 0.821797, 0.820741,
                                   0.819301, 0.81783 , 0.816611, 0.815336, 0.813737, 0.812139, 0.810144,
                                   0.808132, 0.80618 , 0.804365, 0.802627, 0.799118, 0.793286, 0.74658 ,
                                   0.714317, 0.713149, 0.712127, 0.710916, 0.709588, 0.708232, 0.706949,
                                   0.705755, 0.704329, 0.703112, 0.7015  , 0.699942, 0.698148, 0.696534,
                                   0.694879, 0.693193, 0.6914  , 0.689311, 0.687379, 0.684983, 0.680824,
                                   0.674

FigureWidget({
    'data': [{'customdata': array([0.97857 , 0.947649, 0.941255, 0.938699, 0.935492, 0.932175, 0.927678,
                                   0.91917 , 0.834625, 0.832963, 0.831045, 0.827992, 0.826102, 0.822968,
                                   0.820836, 0.817875, 0.815207, 0.812189, 0.80933 , 0.805934, 0.802664,
                                   0.799312, 0.796133, 0.792533, 0.788723, 0.782329, 0.701467, 0.698509,
                                   0.696314, 0.693958, 0.691559, 0.689607, 0.687238, 0.685058, 0.682614,
                                   0.679802, 0.677609, 0.675011, 0.671851, 0.667468, 0.659232, 0.644106,
                                   0.606421, 0.581431, 0.579541, 0.576952, 0.573657, 0.570545, 0.566248,
                                   0.560408, 0.551899, 0.544453, 0.530335, 0.51036 , 0.476159, 0.471232,
                                   0.466264, 0.457793, 0.451314, 0.441615, 0.426717, 0.409925, 0.366387,
                                   0.358

FigureWidget({
    'data': [{'customdata': array([0.94782 , 0.930047, 0.842616, 0.834495, 0.830204, 0.820231, 0.817865,
                                   0.813246, 0.807959, 0.804819, 0.79472 , 0.782843, 0.708691, 0.703788,
                                   0.7002  , 0.696863, 0.693354, 0.689579, 0.685588, 0.677899, 0.666871,
                                   0.593132, 0.588675, 0.582504, 0.578449, 0.574444, 0.566464, 0.555457,
                                   0.518361, 0.495809, 0.467118, 0.461626, 0.456532, 0.416119, 0.395076,
                                   0.362418, 0.353723, 0.312717, 0.292288, 0.254965, 0.206719, 0.165217,
                                   0.119556, 0.102939, 0.085846, 0.054104, 0.026614, 0.022417, 0.      ,
                                   0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
                                   0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
                                   0.   

In [4]:
for datatype, results in all_results.items():
    plot_mat = results.plot_confusion_matrix()
    display(plot_mat)


FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': '3c08baf6-3571-4a09-866c-622d6b63b9c8',
              'x': array([0, 1, 0, 1]),
              'y': array([0, 0, 1, 1])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '06f23ec3-20dc-4478-a6a2-008a8c5d659d',
              'z': array([[  65,    0],
                          [2141,   71]]),
              'zmax': 2141,
              'zmin': 0},
             {'colorbar': {'len': 1, 'lenmode': 'fraction'},

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': 'e25f8f90-1789-4a2b-8d30-427f450bff7f',
              'x': array([0, 1, 0, 1]),
              'y': array([0, 0, 1, 1])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '5816bfb3-9164-4d1a-8fcb-098ae25fbd6b',
              'z': array([[ 187,    0],
                          [1173,  327]]),
              'zmax': 1173,
              'zmin': 0},
             {'colorbar': {'len': 1, 'lenmode': 'fraction'},

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': 'd876153e-e7a0-44c6-b706-9bbd842ffea9',
              'x': array([0, 1, 0, 1]),
              'y': array([0, 0, 1, 1])},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '69d13966-69e5-410a-b375-7f4d50cb238a',
              'z': array([[ 60,   0],
                          [148, 163]]),
              'zmax': 163,
              'zmin': 0},
             {'colorbar': {'len': 1, 'lenmode': 'fraction'},
    